# Generate a Pitch
 - Given the current pitcher, game state, num pitches thrown, opposing batter characteristics, and pitcher characteristics
 - What will the next pitch look like?
## Answers 2 questions:
 - Next pitch type to be thrown?
 - What are the characteristics of the pitch (speed, release pos, zone, etc.)?
 - Will be used as input to the batter
## Potential Difficulties:
 - How do you quantify the change in pitch type distribution based on opposing batters statistics?
 - How do you factor the correlation between pitch characteristics like release pos, etc. to generate realistic pithces?
 - How do you adjust factors based on the opposing batter's zone?
 - How do you adjust based on pitch sequencing?
## TODO:
 - Write script to get features (formatted with labels in torch)
 - Hyperparameter optimization
 - Figure out how to solve small sample..

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

# Define the Generator network
class Generator(nn.Module):
    def __init__(self, input_size, conditioning_size, output_size, hidden_size=128):
        super(Generator, self).__init__()
        self.input_size = input_size
        self.conditioning_size = conditioning_size
        self.output_size = output_size
        
        # Define layers
        self.fc1 = nn.Linear(input_size + conditioning_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()
        
    def forward(self, z, c):
        x = torch.cat([z, c], dim=1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the Discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_size, conditioning_size, hidden_size=128):
        super(Discriminator, self).__init__()
        self.input_size = input_size
        self.conditioning_size = conditioning_size
        
        # Define layers
        self.fc1 = nn.Linear(input_size + conditioning_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        self.activation = nn.LeakyReLU(0.2)
        
    def forward(self, x, c):
        x = torch.cat([x, c], dim=1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the GAN model
class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        
    def forward(self, z, c):
        fake_pitches = self.generator(z, c)
        return fake_pitches

# Generate random noise vector
def generate_noise(batch_size, noise_size):
    return torch.randn(batch_size, noise_size)

# Generate fake conditioning variables (e.g., game factors)
def generate_conditioning(batch_size, conditioning_size):
    return torch.randn(batch_size, conditioning_size)

# Main function for training the GAN
def train_gan(gan, dataloader, num_epochs, batch_size, noise_size, conditioning_size):
    criterion = nn.BCEWithLogitsLoss()
    d_optimizer = optim.Adam(gan.discriminator.parameters(), lr=0.0002)
    g_optimizer = optim.Adam(gan.generator.parameters(), lr=0.0002)
    fixed_noise = generate_noise(1, noise_size)
    fixed_conditioning = generate_conditioning(1, conditioning_size)
    
    for epoch in range(num_epochs):
        for i, (real_data, conditioning) in enumerate(dataloader):
            batch_size = real_data.size(0)
            
            # Train Discriminator
            real_data = real_data.view(-1, gan.generator.output_size)
            real_data = real_data.to(device)
            conditioning = conditioning.to(device)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            
            d_optimizer.zero_grad()
            real_outputs = gan.discriminator(real_data, conditioning)
            d_loss_real = criterion(real_outputs, real_labels)
            
            noise = generate_noise(batch_size, noise_size).to(device)
            fake_pitches = gan.generator(noise, conditioning)
            fake_outputs = gan.discriminator(fake_pitches.detach(), conditioning)
            d_loss_fake = criterion(fake_outputs, fake_labels)
            
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()
            
            # Train Generator
            g_optimizer.zero_grad()
            fake_outputs = gan.discriminator(fake_pitches, conditioning)
            g_loss = criterion(fake_outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()
            
            if i % 100 == 0:
                print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(dataloader)}], "
                      f"Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}")
                
        # Generate sample pitch at end of each epoch
        with torch.no_grad():
            fake_pitch = gan.generator(fixed_noise, fixed_conditioning).cpu().detach().numpy()
            print("Generated Pitch:", fake_pitch)

# Sample usage
if __name__ == "__main__":
    # Define hyperparameters
    batch_size = 32
    noise_size = 100
    conditioning_size = 10
    input_size = 100
    output_size = 13  
    hidden_size = 128
    num_epochs = 10
    
    # Dummy dataset
    pitches = torch.randn(1000, output_size)  # Dummy pitch data
    conditioning_vars = torch.randn(1000, conditioning_size)  # Dummy conditioning variables
    
    
    # Create DataLoader
    dataset = TensorDataset(pitches, conditioning_vars)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Create GAN model
    generator = Generator(input_size, conditioning_size, output_size, hidden_size)
    discriminator = Discriminator(output_size, conditioning_size, hidden_size)
    gan = GAN(generator, discriminator)
    
    # Training the GAN
    train_gan(gan, dataloader, num_epochs, batch_size, noise_size, conditioning_size)


Epoch [0/10], Step [0/32], Discriminator Loss: 1.3925, Generator Loss: 0.7373
Generated Pitch: [[-0.46649253 -0.0610469  -0.20583172 -0.4319409   0.1633176   0.27569598
  -0.2807712  -0.10416228 -0.09181148 -0.01386006  0.32783017 -0.05387577
   0.05743945]]
Epoch [1/10], Step [0/32], Discriminator Loss: 1.3346, Generator Loss: 0.6634
Generated Pitch: [[-0.5472274   0.2023207  -0.00588593 -0.16892159 -0.09596489  0.34547776
   0.04078218 -0.17209241  0.03756853 -0.38079143  0.4800952   0.14270556
   0.1917249 ]]
Epoch [2/10], Step [0/32], Discriminator Loss: 1.2490, Generator Loss: 0.7029
Generated Pitch: [[ 0.13217169  0.21097228  0.74631685  0.4142247  -0.7970775  -0.04869391
   0.27592966  0.28983235  0.19589043  0.04534231  0.17362332  0.47454736
  -0.04931539]]
Epoch [3/10], Step [0/32], Discriminator Loss: 1.3172, Generator Loss: 0.6337
Generated Pitch: [[ 0.9342575  -0.4202156   1.4209844   1.1519558  -0.842765   -0.84077966
  -0.09117113  0.5022152   0.37016445  0.9589467  -0.5

In [4]:
import torch

# Function to generate pitches using the trained GAN model
def generate_pitch(generator, noise_size, conditioning):
    # Generate random noise vector
    noise = torch.randn(1, noise_size)
    
    # Generate pitch using generator
    with torch.no_grad():
        generated_pitch = generator(noise, conditioning)
    
    return generated_pitch

# Sample usage
if __name__ == "__main__":
    # Load the trained GAN model (generator)
    # Replace "generator" with your trained generator model
    generator = Generator(input_size, conditioning_size, output_size, hidden_size)
    generator.load_state_dict(torch.load("generator.pth"))  # Load trained weights
    
    # Define hyperparameters
    noise_size = 100
    conditioning_size = 10
    
    # Define conditioning variables based on game factors
    # Replace this with actual game factors from your MLB simulation model
    conditioning = torch.randn(1, conditioning_size)
    
    # Generate pitch
    generated_pitch = generate_pitch(generator, noise_size, conditioning)
    
    # Print generated pitch
    print("Generated Pitch:", generated_pitch)


FileNotFoundError: [Errno 2] No such file or directory: 'generator.pth'